In [ ]:
import numpy as np 
import sciann as sn 
import time
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from itertools import product, combinations
import scipy.io
#import os
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
# comment this line if you want to run on GPU. 
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # -> exec on CPU

In [ ]:
def axisEqual3D(ax):
        extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
        sz = extents[:,1] - extents[:,0]
        centers = np.mean(extents, axis=1)
        maxsize = max(np.abs(sz))
        r = maxsize/4
        for ctr, dim in zip(centers, 'xyz'):
            getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

# load the snapshot data


In [ ]:
def PrepareData(num_pde=20000, num_data=30000, random=True):

    data = scipy.io.loadmat('data-5s-10hz-2-3-34sensor.mat')
    
    U_star = data['U_star'] # Nt*Nx x 3 x Ns
    XS_star = data['XS_star'] # Nt*Nx x 2
    YT_star = data['YT_star'] # Nx*Ny x 2
    TX_star = data['TX_star']
    XY_star = data['XY_star']
    S_star = data['S_star']
    
    TX1_star = data['TX1_star']
    U_star1 = data['U_star1'] 
    
    c_star = data['c_star']
    rho_star = data['rho_star']
    
    xx = XY_star[:,0:1]
    yy = XY_star[:,1:2]
    
    IDX = np.where((xx == 0.3)| (xx == 0.6))[0]
    c_train = c_star.flatten()[IDX,None]
    rho_train = rho_star.flatten()[IDX,None]
    l_c = IDX.shape[0]
   
    
    tt3 = 0.3 * np.ones(l_c)
    xx3 = xx[IDX,None]
    ss3 = 0.41 * np.ones(l_c)
    yy3 = yy[IDX,None]

    
    N = XS_star.shape[0]
    M = YT_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(XS_star[:,0:1], (1,M)) 
    SS = np.tile(XS_star[:,1:2], (1,M))
    YY = np.tile(YT_star[:,0:1], (1,N))
    TT = np.tile(YT_star[:,1:2], (1,N))
    
    N_data = M*N
    idx6 = np.random.choice(N_data, num_pde, replace=False)
    
    tt2 = TT.flatten()[idx6,None]
    xx2 = XX.flatten()[idx6,None]
    ss2 = SS.flatten()[idx6,None]
    yy2 = YY.flatten()[idx6,None]
    
    L = TX1_star.shape[0]
    T = S_star.shape[0]
    S = S_star.shape[0]
    
    # Rearrange Data 
    TTT = np.tile(TX1_star[:,0:1], (1,T)) # N x S
    XXX = np.tile(TX1_star[:,1:2], (1,T)) # N x S
    SSS = np.tile(S_star, (1,L)).T # N x S
    

    PP = U_star1[:,0,:] # N x S
    UU = U_star1[:,1,:] # N x S
    VV = U_star1[:,2,:] # N x S
    
    # Pick random data.
    
    if random:
        idx = np.random.choice(N_data, num_data, replace=False)
    else:
        s_selected = 2
        maxp = max(np.abs(U_star1[:,1,s_selected]))
        ppp = UU.flatten()
        idx1 = np.where(np.abs(ppp) > 0.001*maxp)[0]
        idx2 = np.where(np.abs(ppp) < 0.001*maxp)[0]
        idx3 = np.random.choice(idx1,int(num_data*0.8),replace=False)
        idx = np.append(idx3, np.random.choice(idx2,int(num_data*0.2),replace=False))

    
    
    tt1 = TTT.flatten()[idx,None]
    xx1 = XXX.flatten()[idx,None]
    ss1 = SSS.flatten()[idx,None]
    yy1 = 0.01 * np.ones(num_data)
    
    
    
    p = PP.flatten()[idx,None] #index of  puv should be consistent with tt1 xx1 ss1 yy1
    u = UU.flatten()[idx,None]
    v = VV.flatten()[idx,None]
    
    
    tt = np.append(np.append(tt3, tt1), tt2)
    xx = np.append(np.append(xx3, xx1), xx2)
    ss = np.append(np.append(ss3, ss1), ss2)
    yy = np.append(np.append(yy3, yy1), yy2)
    
    index_c = np.arange(0,l_c,1)
    index_p = np.arange(0,num_data,1) + l_c
 
    return (tt,xx,ss,yy, u,v,p, c_train, rho_train, index_c, index_p, U_star, U_star1, XS_star, YT_star, TX_star, TX1_star, XY_star, S_star, c_star, rho_star)

In [ ]:
t_train, x_train, s_train, y_train, u_train, v_train, p_train, c_train, rho_train, index_c, index_p, U_star, U_star1, XS_star, YT_star, TX_star, TX1_star, XY_star, S_star, c_star, rho_star = PrepareData(num_pde=30000, num_data=30000, random=False)

# PINN setup 

Consider the 2D acoustic wave equation
$$
\begin{aligned}
&\frac{\partial p}{\partial t} = -c^2(x, z)\rho(x, z)\left(\frac{\partial u}{\partial x} + \frac{\partial v}{\partial z}\right) + f(x, z, t) \\
&\frac{\partial u}{\partial t} = -\frac{1}{\rho(x, z)}\frac{\partial p}{\partial x} \\
&\frac{\partial v}{\partial t} = -\frac{1}{\rho(x, z)}\frac{\partial p}{\partial z},
\end{aligned}
$$
where $p$ is Pressure, $u, v$ are the $x, z-$components of Particle Velocity, $c, \rho$ are Medium Velocity and Density.



Define independent variables with `sn.Variable`:

In [ ]:
from sciann.utils.math import *
ActFunc = 'sin'
ActFunc2 = 'tanh'
hidden_layers = 5*[80]
hidden_layers2 = 5*[40]
dtype = 'float32'

x = sn.Variable("x", dtype=dtype)
y = sn.Variable("y", dtype=dtype)
t = sn.Variable("t", dtype=dtype)
s = sn.Variable("s", dtype=dtype)

Define solution variables with `sn.Functional` (multi-layer neural network):

In [ ]:
p, u, v = sn.Functional(['p','u','v'], [x, y, t, s], hidden_layers, ActFunc)

For inversion, define parameters using `sn.Parameter`:

In [ ]:
c, rho = sn.Functional(['c','rho'], [x, y], hidden_layers2, ActFunc2)

Use `sn.diff` and other mathematical operations to set up the PINN model. 

In [ ]:
c2r = square(c) * rho
r = 1.0 / rho
u_t = sn.diff(u, t)
u_x = sn.diff(u, x)
u_y = sn.diff(u, y)

v_t = sn.diff(v, t)
v_x = sn.diff(v, x)
v_y = sn.diff(v, y)

p_t = sn.diff(p, t)
p_x = sn.diff(p, x)
p_y = sn.diff(p, y)

In [ ]:
# source
f_0 = 10.0*np.pi
alpha = 2.0
M0 = 100.0

def R(t, t_s):
	return M0 * (1-2*(f_0*(t-t_s))**2)*exp(-(f_0*(t-t_s))**2)

def G(x, z, x_s, z_s):
	return exp(-1.0/(alpha**2) * ((x-x_s)**2 + (z-z_s)**2))

def D(x, z, x_s, z_s):
    if ((x == x_s) and (z == z_s)):
        return 1
    else:
        return 0

def f(x, z, t, x_s, z_s, t_s):
	return R(t, t_s)*N(x, z, x_s, z_s)

Define targets (losses) using `sn.Data`, `sn.Tie`, and `sn.PDE` interfaces. 

In [ ]:
# Define constraints 
d1 = sn.Data(p)
d2 = sn.Data(u)
d3 = sn.Data(v)

m1 = sn.Data(c)
m2 = sn.Data(rho)

c1 = sn.Tie(p_t, -c2r * (v_x+u_y) + R(t, 0.1)*G(x,y,s,0.01) )
c2 = sn.Tie(u_t, -r * p_y)
c3 = sn.Tie(v_t, -r * p_x)

Define the optimization model with `sn.SciModel`:

In [ ]:
# Define the optimization model (set of inputs and constraints)
model = sn.SciModel(
    inputs=[x, y, t, s],
    targets=[d1, d2, d3, m1, m2, c1, c2, c3],
    loss_func="mse"
)
#optimizer='scipy-l-BFGS-B',

Prepare the training data according to the order they are defined in `sn.SciModel`. 

In [ ]:
input_data = [x_train, y_train, t_train, s_train]

In [ ]:
data_d1 = p_train
data_d2 = u_train
data_d3 = v_train

data_m1 = c_train
data_m2 = rho_train

data_c1 = 'zeros'
data_c2 = 'zeros'
data_c3 = 'zeros'
target_data = [(index_p, data_d1), (index_p,data_d2), (index_p,data_d3), (index_c,data_m1), (index_c,data_m2), data_c1, data_c2, data_c3]

print(x_train.shape, p_train.shape)
print(index_p.shape, data_d1.shape)

Train the model by calling `.train` function. Check the documentation at www.sciann.com for all the training options. 

In [ ]:
start_time = time.time()
EPOCH = 20000
history = model.train(
    x_true=input_data,
    y_true=target_data,
    epochs=EPOCH,batch_size=1000, shuffle=True,verbose=1,
    adaptive_weights={'method': "NTK", 'freq': 2000, "use_score": True, "alpha": 1.},
    learning_rate={"scheduler": "ExponentialDecay",
                           "initial_learning_rate": 1e-3,
                           "final_learning_rate": 1e-5,
                           "decay_epochs": EPOCH,
                           "verify": True}
)
        
model.save_weights('trained-2layer.hdf5')

elapsed = time.time() - start_time
print('Training time: %.2f minutes' %(elapsed/60.))

In [ ]:
plt.semilogy(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.savefig("./figs/loss.pdf", format='pdf', bbox_inches="tight")

In [ ]:
for loss in history.history:
        np.savetxt("_{}".format("_".join(loss.split("/"))), 
                    np.array(history.history[loss]).reshape(-1, 1))
    
time_steps = np.linspace(0, elapsed, len(history.history["loss"]))
np.savetxt("_Time", time_steps.reshape(-1,1))

In [ ]:
s_selected = 0
p_star = U_star[:,0,s_selected]
s_star = S_star[s_selected]
T = s_star.shape[0]
TT = np.tile(TX_star[:,0:1], (1,T)) # N x T
XX = np.tile(TX_star[:,1:2], (1,T)) # N x T
TT = TT.reshape(600,100)
XX = XX.reshape(600,100)

p_star = p_star.reshape(600,100)

fig, ax = plt.subplots(1, 1, figsize = (5, 5))
#fig.subplots_adjust(right = 1.0)

sc0 = plt.pcolor(XX, TT, (p_star), cmap='gray')
ax.invert_yaxis()
ax.set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax.set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc0)
plt.savefig("./figs/seis0.tiff", format='tif', bbox_inches="tight")

In [ ]:
s_selected = 2
p_star = U_star[:,0,s_selected]
u_star = U_star[:,1,s_selected]
v_star = U_star[:,2,s_selected]
s_star = S_star[s_selected]

N = TX_star.shape[0]
T = s_star.shape[0]

    
# Rearrange Data 
TT = np.tile(TX_star[:,0:1], (1,T)) # N x T
XX = np.tile(TX_star[:,1:2], (1,T)) # N x T
SS = np.tile(s_star, (1,N)).T # N x T

T_plot = TT.flatten()
X_plot = XX.flatten()
Y_plot = 0.01*np.ones(N*T)
S_plot = SS.flatten()

print(S_plot.shape,TT.shape)

XX_plot = XY_star[:,0:1]
YY_plot = XY_star[:,1:2]
M = XX_plot.shape
TT_plot = 0.4*np.ones(M)
SS_plot = 0.21*np.ones(M)


p_pred = p.eval(model,[X_plot, Y_plot, T_plot, S_plot])
u_pred = u.eval(model,[X_plot, Y_plot, T_plot, S_plot])
v_pred = v.eval(model,[X_plot, Y_plot, T_plot, S_plot])

c_pred = c.eval(model, [XX_plot, YY_plot, TT_plot, SS_plot])
rho_pred = rho.eval(model, [XX_plot, YY_plot, TT_plot, SS_plot])

print(max(c_pred),min(c_pred))
print(max(rho_pred),min(rho_pred))
print(max(S_plot),min(S_plot))

print(max(p_star),min(p_star))
print(max(u_star),min(u_star))
print(max(v_star),min(v_star))


In [ ]:
lx = 20
lt = 600#seismogram
llx = 100
llz = 100#model

p_pred = p_pred.reshape(lt,llx)
p_star = p_star.reshape(lt,llx)

u_pred = u_pred.reshape(lt,llx)
u_star = u_star.reshape(lt,llx)

v_pred = v_pred.reshape(lt,llx)
v_star = v_star.reshape(lt,llx)

TT = TT.reshape(lt,llx)
XX = XX.reshape(lt,llx)

XXX = XX_plot.reshape(llx,llz)
YYY = YY_plot.reshape(llx,llz)

import matplotlib.pyplot as plt 

c_pred = c_pred.reshape(llx,llz)
rho_pred = rho_pred.reshape(llx,llz)

import matplotlib as mpl
from matplotlib.cm import cool
from matplotlib.colors import Normalize

from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 2, figsize = (12, 5))
fig.subplots_adjust(right = 1.0)


sc0 = ax[0].pcolor(XXX, YYY, c_pred,  cmap='viridis', vmin=2.0, vmax=3.0)#
ax[0].invert_yaxis()
ax[0].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[0].set_ylabel('z (km)', fontsize=14)
plt.yticks(fontsize=10)
cbar = plt.colorbar(sc0, ax=ax[0])


sc1 = ax[1].pcolor(XXX, YYY, rho_pred,  cmap='viridis', vmin=1.5, vmax=2.0)#
ax[1].invert_yaxis()
plt.xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
plt.ylabel('z (km)', fontsize=14)
plt.yticks(fontsize=10)
cbar = plt.colorbar(sc1, ax=ax[1])
plt.savefig("./figs/pre_model.pdf", format='pdf', bbox_inches="tight")

scipy.io.savemat('c_pred_2layer.mat', {'c_pred':c_pred})
scipy.io.savemat('rho_pred_2layer.mat', {'rho_pred':rho_pred})

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 5))
fig.subplots_adjust(right = 1.0)

sc0 = ax[0].pcolor(XXX, YYY, (c_star), cmap='viridis')
ax[0].invert_yaxis()
ax[0].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[0].set_ylabel('z (km)', fontsize=14)
plt.yticks(fontsize=10)
cbar = plt.colorbar(sc0, ax=ax[0])

sc1 = ax[1].pcolor(XXX, YYY, (rho_star),  cmap='viridis')
ax[1].invert_yaxis()
plt.xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
plt.ylabel('z (km)', fontsize=14)
plt.yticks(fontsize=10)
cbar = plt.colorbar(sc1, ax=ax[1])

plt.savefig("./figs/true-model.pdf", format='pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5,5))

YY = np.arange(0.00,1.0,0.01)

plt.plot(YY, c_star[50,:], linestyle='--', label='Ground truth')
plt.plot(YY, c_pred[50,:], label='PINN')
plt.legend()

plt.xlabel('Depth (km)', fontsize=14)
plt.ylabel('Velocity (km/s)', fontsize=14)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.savefig("./figs/compare_c50.pdf", format='pdf', bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)

sc0 = ax[0].pcolor(XX, TT, (p_pred),cmap='gray', vmin=-0.3747786159933971, vmax=0.4769654096989754)
ax[0].invert_yaxis()
ax[0].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[0].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc0, ax=ax[0])


sc1 = ax[1].pcolor(XX, TT, (u_pred),cmap='gray', vmin=-0.022324010647494202, vmax=0.0476465927714691)
ax[1].invert_yaxis()
ax[1].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[1].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc1, ax=ax[1])

sc2 = ax[2].pcolor(XX, TT, (v_pred), cmap='gray', vmin=-0.13156035880322453, vmax=0.13161169315446972)
ax[2].invert_yaxis()
ax[2].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[2].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc2, ax=ax[2])

plt.savefig("./figs/pre_seis.pdf", format='pdf', bbox_inches="tight")

scipy.io.savemat('p_pred_2layer.mat', {'p_pred':p_pred})
scipy.io.savemat('u_pred_2layer.mat', {'u_pred':u_pred})
scipy.io.savemat('v_pred_2layer.mat', {'v_pred':v_pred})

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)


sc0 = ax[0].pcolor(XX, TT, (p_star), cmap='gray', vmin=-0.3747786159933971, vmax=0.4769654096989754)
ax[0].invert_yaxis()
ax[0].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[0].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc0, ax=ax[0])

sc1 = ax[1].pcolor(XX, TT, (u_star), cmap='gray', vmin=-0.022324010647494202, vmax=0.0476465927714691)
ax[1].invert_yaxis()
ax[1].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[1].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc1, ax=ax[1])

sc2 = ax[2].pcolor(XX, TT, (v_star), cmap='gray', vmin=-0.13156035880322453, vmax=0.13161169315446972)
ax[2].invert_yaxis()
ax[2].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[2].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc2, ax=ax[2])

plt.savefig("./figs/true-seis.pdf", format='pdf', bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)


sc0 = ax[0].pcolor(XX, TT, (np.abs(p_pred - p_star)), cmap='gray')
ax[0].invert_yaxis()
ax[0].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[0].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc0, ax=ax[0])

sc1 = ax[1].pcolor(XX, TT, (np.abs(u_pred - u_star)), cmap='gray')
ax[1].invert_yaxis()
ax[1].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[1].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc1, ax=ax[1])

sc2 = ax[2].pcolor(XX, TT, (np.abs(v_pred - v_star)), cmap='gray')
ax[2].invert_yaxis()
ax[2].set_xlabel('x (km)', fontsize=14)
plt.xticks(fontsize=10)
ax[2].set_ylabel('t (s)', fontsize=14)
plt.yticks(fontsize=10)
plt.colorbar(sc2, ax=ax[2])

plt.savefig("./figs/diff-seis.pdf", format='pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5,5))

YY = np.arange(0.00,0.6,0.001)

plt.plot(YY, p_star[:,60], linestyle='--', label='Ground truth')#extent=[xmin,xmax,zmin,zmax], 
plt.plot(YY, p_pred[:,60], label='PINN')
plt.legend()

plt.xlabel('t (s)', fontsize=14)
#plt.ylabel('Velocity (km/s)', fontsize=14)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.savefig("./figs/compare_p60.pdf", format='pdf', bbox_inches="tight")